In [1]:
import numpy as np 
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('../data/processed_data/MP_Data') 
# Actions that we try to detect
actions = ['hola', 'a', 'b', 'c', 'i', 'n']
key = [1,2,3,4,5,6]
key_action = dict(zip(key,actions))

# number of videos collected 
no_sequences = 12
# Videos are going to be 20 frames in length
sequence_length = 10


### Creation of training and testing data

In [3]:
# dictionary with map of labels: 
labels_dict = {label:num for num, label in enumerate(actions)}

In [34]:
labels_dict

{'hola': 0, 'a': 1, 'b': 2, 'c': 3, 'i': 4, 'n': 5}

In [4]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)), allow_pickle=True)
            window.append(res)
        sequences.append(window)
        labels.append(labels_dict[action])

In [5]:
np.array(sequences).shape

(72, 10, 126)

# creating x and y variables for model training

In [37]:
X = np.array(sequences)

In [38]:
X.shape

(72, 10, 126)

In [39]:
y = to_categorical(labels).astype(int)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [41]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(64, 10, 126)
(8, 10, 126)
(64, 6)
(8, 6)


### build and train LSTM model Neural Network

In [42]:
## allows to monitor accuracy while its training
# check on terminal- into Logs folder:  tensorboard --logdir=. 
# copy link 

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

2022-04-16 19:06:26.323690: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-16 19:06:26.323715: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-16 19:06:26.323748: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [43]:
model = Sequential()
# Long short-term memory 
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(10,126))) # returning sequences for next layer 
model.add(LSTM(128, return_sequences=True, activation='relu')) # returning sequences for next layer 
model.add(LSTM(64, return_sequences=False, activation='relu'))
# Dense layer: each neuron receives input from all the neurons of previous layer
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

In [44]:
# loss categorical_crossentropy because it is a multiclass classification
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [45]:
model.fit(X_train, y_train, steps_per_epoch = 10, epochs=200, callbacks=[tb_callback])

Epoch 1/200
 2/10 [=====>........................] - ETA: 0s - loss: 1.7912 - categorical_accuracy: 0.2500 

2022-04-16 19:06:30.277597: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-16 19:06:30.277621: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


10/10 [==============================] - 4s 58ms/step - loss: 1.7924 - categorical_accuracy: 0.2278
Epoch 2/200
 1/10 [==>...........................] - ETA: 0s - loss: 1.7850 - categorical_accuracy: 0.2857

2022-04-16 19:06:30.481270: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-04-16 19:06:30.503367: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-04-16 19:06:30.524668: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_04_16_19_06_30
2022-04-16 19:06:30.539613: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to Logs/train/plugins/profile/2022_04_16_19_06_30/GG-McAir.local.trace.json.gz
2022-04-16 19:06:30.553396: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_04_16_19_06_30
2022-04-16 19:06:30.553753: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to Logs/train/plugins/profile/2022_04_16_19_06_30/GG-McAir.local.memory_profile.json.gz
2022-04-16 19:06:30.556955:

10/10 [==============================] - 0s 15ms/step - loss: 1.7839 - categorical_accuracy: 0.1972
Epoch 3/200
10/10 [==============================] - 0s 20ms/step - loss: 1.7770 - categorical_accuracy: 0.3178
Epoch 4/200
10/10 [==============================] - 0s 21ms/step - loss: 1.7685 - categorical_accuracy: 0.2418
Epoch 5/200
10/10 [==============================] - 0s 23ms/step - loss: 1.7428 - categorical_accuracy: 0.3011
Epoch 6/200
10/10 [==============================] - 0s 20ms/step - loss: 1.6555 - categorical_accuracy: 0.2550
Epoch 7/200
10/10 [==============================] - 0s 28ms/step - loss: 1.8468 - categorical_accuracy: 0.1805
Epoch 8/200
10/10 [==============================] - 0s 26ms/step - loss: 1.6626 - categorical_accuracy: 0.2617
Epoch 9/200
10/10 [==============================] - 0s 23ms/step - loss: 1.5587 - categorical_accuracy: 0.3892
Epoch 10/200
10/10 [==============================] - 0s 24ms/step - loss: 1.4858 - categorical_accuracy: 0.4320
Epo

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 64)            48896     
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 203,558
Trainable params: 203,558
Non-trainable params: 0
________________________________________________

In [47]:
X_test[1]

array([[ 0.        ,  0.        ,  0.        , ...,  0.43856904,
         0.23973034, -0.06063496],
       [ 0.        ,  0.        ,  0.        , ...,  0.43856904,
         0.23973034, -0.06063496],
       [ 0.        ,  0.        ,  0.        , ...,  0.43856904,
         0.23973034, -0.06063496],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.43856904,
         0.23973034, -0.06063496],
       [ 0.        ,  0.        ,  0.        , ...,  0.43856904,
         0.23973034, -0.06063496],
       [ 0.        ,  0.        ,  0.        , ...,  0.43856904,
         0.23973034, -0.06063496]])

In [48]:
res = model.predict(X_test)

# to check some values 
actions = np.array(actions)
print(actions[np.argmax(res[4])])
print(actions[np.argmax(y_test[4])])

i
a


In [49]:
# model.save('video_model7_weights')

In [50]:
X_test

array([[[ 0.        ,  0.        ,  0.        , ...,  0.38093248,
          0.56406164, -0.0666336 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.38093248,
          0.56406164, -0.0666336 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.38093248,
          0.56406164, -0.0666336 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.38093248,
          0.56406164, -0.0666336 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.38093248,
          0.56406164, -0.0666336 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.38093248,
          0.56406164, -0.0666336 ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.43856904,
          0.23973034, -0.06063496],
        [ 0.        ,  0.        ,  0.        , ...,  0.43856904,
          0.23973034, -0.06063496],
        [ 0.        ,  0.        ,  0.        , ...,  0.43856904,
          0.23973034, -0.06063496],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [51]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [52]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[6, 0],
        [2, 0]],

       [[6, 0],
        [2, 0]],

       [[4, 2],
        [0, 2]],

       [[5, 2],
        [0, 1]],

       [[7, 0],
        [0, 1]]])

In [53]:
accuracy_score(ytrue, yhat)

0.5

In [2]:
import numpy as np

In [33]:
np.load('../data/processed_data/MP_Data/bien/1/0.npy')

ValueError: Object arrays cannot be loaded when allow_pickle=False